In [1]:
# This notebook aggregates all output from toboggan for an input dataset
# and then creates a runtime histogram for that dataset

# CHANGE froot TO THE DATASET YOU WANT: zebra, human, mouse
froot = 'mouse'
timeoutexceed = 50
which_alg = "toboggan"

In [2]:
inputfile = "all-" + froot + ".txt"

import table_generator

datadict, datamatrix = table_generator.make_tables(inputfile)

In [9]:
info_dict = table_generator.get_toboggan_timing_info(datadict, datamatrix)
num_trivial = info_dict['num_trivial']
num_timedout = info_dict['num_timedout']
time_totals = info_dict['time_totals']
total_num = info_dict['total_num']

NameError: name 'get_toboggan_timing_info' is not defined

In [4]:
table_generator.print_data_summary( froot, len(datadict), num_trivial )
table_generator.print_alg_summary( which_alg, timeoutexceed, num_timedout, len(time_totals) )

print(num_timedout + len(time_totals))

mouse has 
	total   instances:      1316056
	trivial instances:       842873
	nontriv instances:       473183

toboggan with 50s timeout has
	timeout instances:         2487
	success instances:       470696
473183


In [5]:
import matplotlib.pyplot as plt
import numpy as np

plt.clf()
#logbins=np.logspace(-4, 3, 7)
logbins = [10**-4, 10**-3, 10**-2, 10**-1, 1, 10, 100, 1000, 10000]
n, bins, patchs = plt.hist(time_totals, bins=logbins, log=True)
plt.title("Runtimes: {}, {}".format(which_alg, froot))
plt.xlabel("time")
plt.ylabel("Frequency")
plt.gca().set_xscale("log")

#plt.show()
plt.savefig( ''.join(['runtime-toboggan-', froot, '.eps']), format='eps', dpi=300)
plt.savefig( ''.join(['runtime-toboggan-', froot, '.png']), format='png', dpi=300)
plt.close()

In [6]:
len(datamatrix)

1316056

In [7]:
len(datadict)

1316056